In [2]:
from bs4 import BeautifulSoup
import json
import csv
import requests
import random
import re

Getting the institue links, we need those to scrape info about schools.

In [47]:
institute_link = []
link_ids = {}

with open("data/infoscuola-database.json","r") as f:
    for school_info in file_data['data']:

        institute_name = school_info[0].split('"')
        link_info = institute_name[1].split("/")
        id_link = link_info[-1]
        
        if(id_link not in link_ids):
            institute_link.append(institute_name[1])
            link_ids[id_link] = True
        

Getting school links

In [58]:
# id to look for in the HTML: table-unita-scolastiche
schools_link = []

for link in institute_link:
    response = requests.get(link)
    soup = BeautifulSoup(response.content,"html.parser")
    table = soup.find("table",attrs={'id':'table-unita-scolastiche'})
    tbody = table.find("tbody")
    a_tags = tbody.findAll(["a"],href=True)

    for a_tag in a_tags:
        schools_link.append("https://www.vivoscuola.it"+a_tag['href'])
    random.uniform(0.9,1.2)
    

In [59]:
with open('data/schoo_links.txt', 'w') as f:
    for line in schools_link:
        f.write(f"{line}\n")


We now parse school details.

In [ ]:
schools_link = []
with open("data/schoo_links.txt","r") as f:
    schools_link = f.readlines()

schools_details = []

for link in schools_link:
    response = requests.get(link)
    soup = BeautifulSoup(response.content,"html.parser")

    entry = [None,None,None,None,None,None,None,None,None,None]
    school_info = {
        "Tipo scuola":[0,None],
        "Indirizzo":[0,None],
        "Telefono":[0,None],
        "Fax":[0,None],
        "Email istituto":[0,None],
        "Email dirigenza":[0,None],
        "Email segreteria":[0,None],
        "Sito web":[0,None],
        "Codice MIUR":[0,None],
    }
    title = soup.find("div",attrs={'class':'u-content-title'})
    entry[9] = title.text
            
    main_div = soup.find("div",attrs={'class':"Grid Prose u-padding-top-xxl"})
        
    info_descriptions = main_div.select("div[class*=u-textItalic]")

    for index,info in enumerate(info_descriptions):
        school_info[info.text.strip()] = [index,info.text.strip()]
    general_infos = main_div.select("div[class*=u-size3of4]")

    for index,key in enumerate(school_info):
        if(school_info[key][1] != None):
            entry[index] = general_infos[school_info[key][0]].text.strip()
    
    
    schools_details.append(entry)
    random.uniform(0.9,1.4)
    
#scripts = soup.find_all("script")
#pattern = re.compile('var provincecode (.*?)',re.MULTILINE)
#for script in scripts:
#    print(script)
#    if(pattern.match(str(script.string))):
#        data = pattern.match(script.string)
#        print(data)
#    break

    

we now clean the scraped data and save it on a .csv file

In [44]:
for school in schools_details:
    school[9] = ''.join(school[9].splitlines())
    school[1] = ''.join(school[1].splitlines())

    for item in school:
        if(item != None):
            item = re.sub(' +', ' ', item)

with open("data/school_list.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["school_type","address","phone_number","fax_number","school_email","management_email","office_email","website","miur_code","school_name"])
    writer.writerows(schools_details)

